In [ ]:
#SBERT-FAISS -> LDA & genre1 KNN

#기본 세팅

In [ ]:
pip install xlrd

In [ ]:
pip install sentence-transformers faiss-cpu

In [ ]:
pip install konlpy

In [ ]:
import pandas as pd
import requests
import re
import konlpy
from konlpy.tag import Okt
from gensim import corpora
from gensim.corpora import Dictionary
from gensim.models import LdaModel

#데이터 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


원본데이터

In [ ]:
df_original = pd.read_csv("/content/drive/MyDrive/알라딘 베스트셀러 Top 1000 1999~2024/가공 데이터/추천서비스_최종데이터(10.26).csv",
            encoding = 'utf-8-sig',
            dtype = {'ISBN' : "str"})
df = df_original.copy()

#CID group 하기

In [ ]:
cid_data = pd.read_excel("/content/drive/MyDrive/알라딘 베스트셀러 Top 1000 1999~2024/가공 데이터/aladin_Category_CID_20210927.xls")

#열 재정의, 필요없는 행 삭제
cid_data.columns = cid_data.iloc[1]
cid_data = cid_data.drop(index = [0,1])
cid_data.head(3)

1,CID,카테고리명,몰,1Depth,2Depth,3Depth,4Depth,5Depth
2,1230,가정/요리/뷰티,국내도서,가정/요리/뷰티,NaN,NaN,NaN,NaN
3,90452,가계부,국내도서,가정/요리/뷰티,가계부,NaN,NaN,NaN
4,53471,건강요리,국내도서,가정/요리/뷰티,건강요리,NaN,NaN,NaN


In [ ]:
grouped_cid = cid_data.groupby(['1Depth']).agg({'CID': lambda x: list(x)}).reset_index()

print("[1Depth] groupby 전체 length : ", len(grouped_cid))
print("CID가 하나만 속하는 데이터 :", len(grouped_cid[grouped_cid['CID'].apply(len) == 1]))
grouped_cid[50:60]

[1Depth] groupby 전체 length :  111
CID가 하나만 속하는 데이터 : 11


1,1Depth,CID
50,사전/기타,"[38419, 39068, 41818, 41819, 39200, 42527, 425..."
51,사회과학,"[798, 51052, 50999, 51121, 51117, 51111, 69510..."
52,상품권,[28640]
53,성인,[6548]
54,소설/시/희곡,"[1, 50930, 89481, 89482, 50922, 50927, 51116, ..."
55,수험서,"[90843, 97459, 97460, 97461, 97462, 97463, 974..."
56,수험서/자격증,"[1383, 34582, 34612, 34613, 34625, 34626, 3462..."
57,스페인 도서,"[28254, 159297, 75161, 75167, 75174, 75172, 75..."
58,알라딘오디오북,"[158585, 167553, 175974, 167547, 176194, 16754..."
59,애니메이션,"[6550, 69740, 55652, 56376, 89266, 66606, 6973..."


In [ ]:
#빠른 매핑을 위한 딕셔너리 만들기
depth2_dict = {}
for i, items in enumerate(grouped_cid['CID']):
  for id in items:
    depth2_dict[id] = grouped_cid['1Depth'].index[i]

#적용
df['genre_code'] = df['genre_id'].map(depth2_dict).astype('Int32')
df['genre_code'] = df['genre_code'].fillna(0).astype('Int32')
df.head(1)

,title,author,genre_id,genre,genre2,ISBN,description_aladin,description_nouns,salesPoint,mallType,lda_result,description_naver,genre_code,요약,추천단어
0,국어사개설 - 신정판,이기문 (지은이),8292,국내도서>대학교재/전문서적>어문학계열>국어국문학>국어사/방언론,대학교재/전문서적,9788976263728,국어의 계통과 형성 문자체계를 먼저 살피고 이어 고대국어 전기 중세국어와 후기 ...,국어 계통 형성 문자체계 먼저 살피 고대 국어 전기 중세 국어 후기 중세 국어 근대...,5758,BOOK,38,"국어의 계통과 형성, 문자체계를 먼저 살피고 이어 고대국어, 전기 중세국어와 후기 ...",40,"해당 텍스트는 국어의 발전 과정에 대해 계통, 문자체계부터 시작하여 고대국어부터 현...",과정 발전 문자체계 설명 문서 계통 해당 국어 시대 시작 텍스트


In [ ]:
#test : 분류에 쓰일 데이터들
#new : 새로운 사용자 기록

df_test = df[['ISBN','genre_code', 'lda_result', '요약']]

#도서 검색
new_data에 [group 된 CID, lda 결과, 요약된 책 소개]

In [ ]:
#검색어
query = "습관"

In [ ]:
TTBKey = "ttbleejunho09091938001"

url = f"http://www.aladin.co.kr/ttb/api/ItemSearch.aspx?ttbkey={TTBKey}&Query={query}&QueryType=Title&MaxResults=10&start=1&SearchTarget=Book&output=JS&Version=20131101"
res = requests.get(url)
res.raise_for_status()

original_cid = res.json()['item'][0]['categoryId']
original_description = res.json()['item'][0]['description']

In [ ]:
#1. CID
grouped_cid = depth2_dict[original_cid]

In [ ]:
#2. LDA
#new_description 전처리
new_description = re.sub(r'[^\w\s가-힣]', ' ', original_description)  # 특수문자 제거
new_description = re.sub(r'[\n\d]', ' ', new_description)  # 줄바꿈, 숫자 제거

#Okt, 불용어사전 준비
Okt = konlpy.tag.Okt()
stopword_data_path = '/content/drive/MyDrive/알라딘 베스트셀러 Top 1000 1999~2024/가공 데이터/stopwords-ko.txt'
with open(stopword_data_path, 'r', encoding='utf-8') as f:
  stopwords = [line.strip() for line in f.readlines()]

#Okt, 불용어제거 실행
Okt_morphs = Okt.pos(new_description)
new_description = ' '.join([word for word, pos in Okt_morphs if pos == 'Noun' and word not in stopwords and len(word)>1])
new_description

'서로 상처 주지 정도 대화 대화 코칭 전문가 하야시 겐타 아무 상처 화법 문제 대한 해결 제시'

In [ ]:
# 저장된 LDA 모델을 불러오기
lda_model = LdaModel.load('/content/drive/MyDrive/알라딘 베스트셀러 Top 1000 1999~2024/가공 데이터/lda model/lda_model.model')

# 저장된 dictionary 파일을 불러오기
dictionary = Dictionary.load('/content/drive/MyDrive/알라딘 베스트셀러 Top 1000 1999~2024/가공 데이터/lda model/dictionary.dict')

# 저장된 corpus 파일을 불러오기
corpus = corpora.MmCorpus('/content/drive/MyDrive/알라딘 베스트셀러 Top 1000 1999~2024/가공 데이터/lda model/corpus.mm')

In [ ]:
new_doc = new_description.split() #토큰화
new_doc2bow = dictionary.doc2bow(new_doc) #doc2bow
new_doc_lda = lda_model.get_document_topics(new_doc2bow) #lda 분석

for topic_id, prob in new_doc_lda:
    print(f"토픽 {topic_id}: 확률 {prob}")
#최대 확률 토픽
max_topic_id = max(new_doc_lda, key=lambda x: x[1])[0]

lda_topic = max_topic_id #저장
print(lda_topic)

토픽 47: 확률 0.18505416810512543
토픽 56: 확률 0.08605854958295822
토픽 95: 확률 0.6432255506515503
95


In [ ]:
new_data = [grouped_cid, lda_topic, original_description]
print(new_data)

[81, 95, '어떻게 해야 서로에게 상처 주지 않으면서 할 말은 어느 정도 하는 대화를 나눌 수 있을까? 대화 코칭 전문가, 하야시 겐타로의 『아무도 상처받지 않는 대화법』은 바로 이런 문제에 대한 해결책을 제시하는 책이다.']


#책 소개 SBERT - FAISS 분석
책 소개 데이터 Sentence BERT로 임베딩 후 FAISS분석

S-BERT로 임베딩

In [ ]:
import tensorflow as tf
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

# 2. Sentence-BERT 모델 로드
model = SentenceTransformer('all-MiniLM-L6-v2')  # 임베딩용 모델 선택

"""
# 3. 임베딩 생성 (정수 결합 + 문장 임베딩)
def create_embedding(data):
    sentence_embedding = model.encode(data)  # 문장 임베딩

    return sentence_embedding

# 전체 데이터 임베딩 생성
embeddings = create_embedding(df_test['요약'])

저장
print(embeddings.shape, df_test.shape)
np.save("/content/drive/MyDrive/알라딘 베스트셀러 Top 1000 1999~2024/가공 데이터/s_bert_embeddings.npy", embeddings)"""

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


"\n# 3. 임베딩 생성 (정수 결합 + 문장 임베딩)\ndef create_embedding(data):\n    sentence_embedding = model.encode(data)  # 문장 임베딩\n\n    return sentence_embedding\n\n# 전체 데이터 임베딩 생성\nembeddings = create_embedding(df_test['요약'])"

FAISS 사용

In [ ]:
#embeddings 불러오기
embeddings = np.load("/content/drive/MyDrive/알라딘 베스트셀러 Top 1000 1999~2024/가공 데이터/s_bert_embeddings.npy")

In [ ]:
# 4. FAISS 인덱스 생성
d = embeddings.shape[1]  # 벡터 차원
index = faiss.IndexFlatL2(d)  # L2 거리 기반 인덱스 생성
index.add(embeddings)  # 임베딩 데이터 추가

# 5. 새로운 데이터로 유사한 데 이터 검색
def find_similar_data(sentence, top_k=1):
  query_embedding = model.encode(sentence).reshape(1,-1) #-1은 자동으로 계산하라는 뜻.

  _, similar_indices = index.search(query_embedding, top_k)  # 유사도 검색
  return similar_indices

# 테스트
similar_data = find_similar_data(new_data[2], top_k=100)

for i in similar_data[0]:
  print(df_test.iloc[i])

ISBN                                             9788960970274 
genre_code                                                   91
lda_result                                                   94
요약            이 책은 선교사의 호피족 인디언 사역 경험을 토대로 하나님의 채워주심에 대한 비밀을...
Name: 14166, dtype: object
ISBN                                             9788970943015 
genre_code                                                   73
lda_result                                                   50
요약            이 책은 비가 오는 날 아이들이 답답해하는 이야기로, 호호 할머니와 강아지, 아기 ...
Name: 7112, dtype: object
ISBN                                             9788970577012 
genre_code                                                   61
lda_result                                                   51
요약            어린이들을 위한 문학책으로, 학습서에 많이 노출되는 요즘 어린이들이 문학의 상상력과...
Name: 9358, dtype: object
ISBN                                             9788949110547 
genre_code                                                   73
lda_result               

In [ ]:
df_test_sbert_faiss = df_test.iloc[similar_data[0]]
df_test_sbert_faiss.head(3)

,ISBN,genre_code,lda_result,요약
14166,9788960970274,91,94,이 책은 선교사의 호피족 인디언 사역 경험을 토대로 하나님의 채워주심에 대한 비밀을...
7112,9788970943015,73,50,"이 책은 비가 오는 날 아이들이 답답해하는 이야기로, 호호 할머니와 강아지, 아기 ..."
9358,9788970577012,61,51,"어린이들을 위한 문학책으로, 학습서에 많이 노출되는 요즘 어린이들이 문학의 상상력과..."


#KNN - genre_code, lda_result
genre_code, lda_result 이용한 knn 유사도로 비슷한거 나열

In [ ]:
from sklearn.neighbors import NearestNeighbors

classifier = NearestNeighbors(n_neighbors = 10, metric='euclidean').fit(df_test_sbert_faiss[['lda_result','genre_code']])
distances, indices = classifier.kneighbors([new_data[:2]])

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


In [ ]:
print("유사한 항목 (가까운 순서대로):")
result = pd.DataFrame(columns=df_test_sbert_faiss.columns)

for idx, distance in zip(indices[0], distances[0]):
  print(f"인덱스: {idx}, 거리: {distance}\n데이터: {df_test.iloc[idx]}\n\n")

  # 유사한 데이터의 행 가져오기
  similar_row = df_test_sbert_faiss.iloc[idx]

  # result DataFrame에 행 추가
  result = pd.concat([result, similar_row.to_frame().T], ignore_index=True)



유사한 항목 (가까운 순서대로):
인덱스: 0, 거리: 13.601470508735444
데이터: ISBN                                             9788976263728 
genre_code                                                   40
lda_result                                                   38
요약            해당 텍스트는 국어의 발전 과정에 대해 계통, 문자체계부터 시작하여 고대국어부터 현...
Name: 0, dtype: object


인덱스: 82, 거리: 14.142135623730951
데이터: ISBN                                             9788985548458 
genre_code                                                   51
lda_result                                                   19
요약             인물과 사상  제11권  주제는 권위주의이다  이 책에서는 특히 강준만 교수의 손...
Name: 82, dtype: object


인덱스: 57, 거리: 14.317821063276353
데이터: ISBN                                             9788990274069 
genre_code                                                   51
lda_result                                                   19
요약            지은이가 8년 동안 격월간  녹색평론 을 엮어내는 동안 틈틈이 쓰거나 말했던 기록들...
Name: 57, dtype: object


인덱스: 25, 거리: 18.7882942280

In [ ]:
result

,ISBN,genre_code,lda_result,요약
0,9788960970274,91,94,이 책은 선교사의 호피족 인디언 사역 경험을 토대로 하나님의 채워주심에 대한 비밀을...
1,9788990220868,97,95,청소년 범죄자가 대자연에서 스피릿베어를 만나며 분노와 용서의 여정을 거듭합니다.작가...
2,9788934902690,92,95,내 뜻을 따라주지 않는 아이 사춘기라서 반항적인 아이 공부에는 관심없고 놀기만 ...
3,9788932010250,78,89,"에피쿠로스의 전체 저작을 모아놓은 현존하는 모든 글을 소개하는 사상 입문서로, 쾌락..."
4,9788974271992,78,89,사랑을 선택하는 특별한 기준 은 무엇이어야 하며 정신적 안정은 어디에서 구해지는...
5,9788984051218,81,95,저자는 어떻게 해야 청산유수로 말을 잘할 수 있는지만 강조하지 않는다 저자가 이 ...
6,9788901252520,81,95,신뢰할 수 있는 심리학 지식과 회복력을 키우는 방법을 제시하는 다정하고 따뜻한 심리...
7,9788995098400,91,60,절에 가면 돌 하나 건물 그림에 담긴 뜻을 모르고 그저 보기만 하는 경우가 흔하...
8,9791165733766,73,82,은색 사자에 관한 거짓 소문과 확인 없이 소문을 퍼트린 동물들에 대해 이야기하는 책...
9,9788984887473,73,67,이 그림책은 아이들에게 엄마의 사랑과 안정감을 전하는 이야기를 담고 있습니다. 아기...


In [ ]:
new_data

[81,
 95,
 '어떻게 해야 서로에게 상처 주지 않으면서 할 말은 어느 정도 하는 대화를 나눌 수 있을까? 대화 코칭 전문가, 하야시 겐타로의 『아무도 상처받지 않는 대화법』은 바로 이런 문제에 대한 해결책을 제시하는 책이다.']